In [15]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

In [16]:
df = pd.read_csv('online_shoppers_intention (1).csv')

In [19]:
cat_cols = [col for col in df.columns if df[col].dtypes in ['object', 'bool'] and col not in ['Revenue']]

In [20]:
for col in cat_cols:
    le = LabelEncoder()
    df.loc[:, col] = le.fit_transform(df[col])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_24648\1663346978.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col] = le.fit_transform(df[col])
C:\Users\lenovo\AppData\Local\Temp\ipykernel_24648\1663346978.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, col] = le.fit_transform(df[col])


In [17]:
num_cols = [col for col in df.columns if df[col].dtypes in ['int64', 'float64'] and col not in ['Revenue']]

In [18]:
for col in num_cols:
    scaler = StandardScaler()
    df.loc[:, col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

In [9]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates in %,ExitRates in %,PageValues,SpecialDay (probability),Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,2,-1.233426,-0.790293,-0.894178,-0.762629,2,-0.550552,False
1,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.317178,-0.308821,2,-0.136078,-0.207952,-0.894178,-0.514182,2,-0.550552,False
2,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,2,2.058618,-0.790293,2.437081,-0.265735,2,-0.550552,False
3,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.622954,0.573535,1.994610,-0.317178,-0.308821,2,0.961270,-0.207952,-0.477771,-0.017289,2,-0.550552,False
4,-0.696993,-0.457191,-0.396478,-0.244931,-0.488636,-0.296430,-0.045196,0.142551,-0.317178,-0.308821,2,0.961270,0.374389,-0.894178,-0.017289,2,1.816360,False


In [13]:
df['Revenue'] = pd.Categorical(df['Revenue'])

In [21]:
X = df.drop('Revenue', axis=1)
y = df['Revenue']

In [12]:
y.value_counts()

False    10422
True      1908
Name: Revenue, dtype: int64

In [22]:
n_splits = 2
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [105]:
class_weight = {0: 1.183, 1: 6.462}
class_weight1 = {0: 0.400, 1: 1.600}

In [106]:
model = LogisticRegression(solver='liblinear', penalty='l2', 
                           class_weight=class_weight, random_state=1)
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [107]:
print(confusion_matrix(preds, y_test))

[[1859  101]
 [ 225  281]]


In [108]:
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1960
           1       0.74      0.56      0.63       506

    accuracy                           0.87      2466
   macro avg       0.81      0.75      0.78      2466
weighted avg       0.86      0.87      0.86      2466



In [109]:
from sklearn.utils.class_weight import compute_class_weight

In [110]:
sklearn_weights1 = compute_class_weight(class_weight=None, y=df['Revenue'], classes=np.unique(y))
sklearn_weights1

array([1., 1.])

In [111]:
print(f'The weights for the majority class is {sklearn_weights1[0]*2:.3f}')
print(f'The weights for the minority class is {sklearn_weights1[1]*2:.3f}')

The weights for the majority class is 2.000
The weights for the minority class is 2.000


In [112]:
sklearn_weights2 = compute_class_weight(class_weight='balanced', y=df['Revenue'], classes=np.unique(y))
sklearn_weights2

array([0.59153713, 3.23113208])

In [113]:
print(f'The weights for the majority class is {sklearn_weights2[0]*2:.3f}')
print(f'The weights for the minority class is {sklearn_weights2[1]*2:.3f}')

The weights for the majority class is 1.183
The weights for the minority class is 6.462


In [114]:
dict1={0: 0.2, 1: 0.8}
sklearn_weights3 = compute_class_weight(class_weight=dict1, y=df['Revenue'], classes=np.unique(y))
sklearn_weights3

array([0.2, 0.8])

In [115]:
print(f'The weights for the majority class is {sklearn_weights3[0]*2:.3f}')
print(f'The weights for the minority class is {sklearn_weights3[1]*2:.3f}')

The weights for the majority class is 0.400
The weights for the minority class is 1.600


In [ ]:
!pip install supervised

In [23]:
from supervised.automl import AutoML

In [24]:
automl = AutoML(eval_metric='logloss')
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

AutoML directory: AutoML_3
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline logloss 0.431274 trained in 0.54 seconds
2_DecisionTree logloss 0.219619 trained in 14.59 seconds
3_Linear logloss 0.264325 trained in 4.91 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost logloss 0.212802 trained in 5.21 seconds
5_Default_NeuralNetwork logloss 0.231329 trained in 2.74 seconds
6_Default_RandomForest logloss 0.207766 trained in 6.34 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.206185 trained in 2.15 seconds
AutoML fit time: 44.71 seconds
AutoML best model: Ensemble


In [31]:
automl = AutoML(algorithms=["Xgboost", "CatBoost", "Random Forest", "LightGBM", "Neural Network"], 
                start_random_models=5)
automl.fit(X_train, y_train)

predictions = automl.predict_all(X_test)

AutoML directory: AutoML_7
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Xgboost', 'CatBoost', 'Random Forest', 'LightGBM', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 5 models
1_Default_LightGBM logloss 0.212129 trained in 29.54 seconds
2_Default_Xgboost logloss 0.212802 trained in 5.19 seconds
3_Default_CatBoost logloss 0.206703 trained in 2.9 seconds
4_Default_NeuralNetwork logloss 0.243113 trained in 2.95 seconds
5_Default_RandomForest logloss 0.206768 trained in 4.97 seconds
* Step not_so_random will try to check up to 20 models
10_LightGBM logloss 0.205878 trained in 4.54 seconds
6_Xgboost logloss 0.209067 trained in 5.15 seconds
14_CatBoost logloss 0.204663 trained in 4.43 seconds
18_RandomForest logloss 0.217458 trained in 

In [35]:
print(confusion_matrix(predictions['label'], y_test))

[[4825  274]
 [ 386  680]]


In [34]:
print(classification_report(predictions['label'], y_test))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      5099
           1       0.71      0.64      0.67      1066

    accuracy                           0.89      6165
   macro avg       0.82      0.79      0.80      6165
weighted avg       0.89      0.89      0.89      6165



In [28]:
automl = AutoML(
    algorithms=["Xgboost", "CatBoost", "Random Forest", "LightGBM", "Neural Network"],
    start_random_models=10,
    hill_climbing_steps=5,
    top_models_to_improve=3,
    golden_features=True,
    features_selection=False,
    stack_models=True,
    train_ensemble=True,
    explain_level=0,
    validation_strategy={
        "validation_type": "kfold",
        "k_folds": 10,
        "shuffle": True,
        "stratify": False,
    }
)

automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

AutoML directory: AutoML_5
The task is binary_classification with evaluation metric logloss
AutoML will use algorithms: ['Xgboost', 'CatBoost', 'Random Forest', 'LightGBM', 'Neural Network']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'hill_climbing_1', 'hill_climbing_2', 'hill_climbing_3', 'hill_climbing_4', 'hill_climbing_5', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 5 models
1_Default_LightGBM logloss 0.228981 trained in 7.72 seconds
2_Default_Xgboost logloss 0.229131 trained in 5.34 seconds
3_Default_CatBoost logloss 0.228626 trained in 9.63 seconds
4_Default_NeuralNetwork logloss 0.310537 trained in 11.68 seconds
5_Default_RandomForest logloss 0.232287 trained in 10.81 seconds
* Step not_so_random will try to check up to 45 models
15_LightGBM logloss 0.224341 tr

117_RandomForest logloss 0.223835 trained in 27.46 seconds
118_LightGBM logloss 0.224354 trained in 17.52 seconds
119_RandomForest logloss 0.225989 trained in 27.4 seconds
120_Xgboost logloss 0.227278 trained in 18.1 seconds
121_CatBoost logloss 0.228176 trained in 27.92 seconds
122_CatBoost logloss 0.229743 trained in 29.4 seconds
123_CatBoost logloss 0.228945 trained in 27.21 seconds
124_NeuralNetwork logloss 0.264616 trained in 23.3 seconds
125_NeuralNetwork logloss 0.274591 trained in 24.48 seconds
126_NeuralNetwork logloss 0.263501 trained in 23.14 seconds
* Step ensemble will try to check up to 1 model
Ensemble logloss 0.220751 trained in 103.51 seconds
* Step stack will try to check up to 50 models
117_RandomForest_Stacked logloss 0.222487 trained in 94.5 seconds
105_LightGBM_Stacked logloss 0.226212 trained in 17.21 seconds
98_Xgboost_Stacked logloss 0.225722 trained in 18.34 seconds
110_CatBoost_Stacked logloss 0.224003 trained in 42.06 seconds
126_NeuralNetwork_Stacked loglos

In [39]:
import xgboost as xgb
import catboost as cat
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

In [40]:
models = [('rf', RandomForestClassifier()), ('xgboost',xgb.XGBClassifier()), ('catboost', cat.CatBoostClassifier()),
          ('lightgbm', lgb.LGBMClassifier())]
stacking = StackingClassifier(estimators=models)

In [42]:
stacking.fit(X_train, y_train)

Learning rate set to 0.022399
0:	learn: 0.6621612	total: 16.5ms	remaining: 16.4s
1:	learn: 0.6392112	total: 20ms	remaining: 9.96s
2:	learn: 0.6164118	total: 23.3ms	remaining: 7.75s
3:	learn: 0.5950460	total: 26.4ms	remaining: 6.57s
4:	learn: 0.5743934	total: 29.6ms	remaining: 5.89s
5:	learn: 0.5565941	total: 32.7ms	remaining: 5.42s
6:	learn: 0.5369229	total: 35.7ms	remaining: 5.07s
7:	learn: 0.5192749	total: 38.7ms	remaining: 4.79s
8:	learn: 0.5023724	total: 41.6ms	remaining: 4.58s
9:	learn: 0.4877099	total: 44.5ms	remaining: 4.41s
10:	learn: 0.4740961	total: 47.5ms	remaining: 4.27s
11:	learn: 0.4609925	total: 50.4ms	remaining: 4.15s
12:	learn: 0.4496503	total: 53.4ms	remaining: 4.06s
13:	learn: 0.4375974	total: 56.6ms	remaining: 3.99s
14:	learn: 0.4272164	total: 59.6ms	remaining: 3.91s
15:	learn: 0.4167573	total: 62.8ms	remaining: 3.86s
16:	learn: 0.4076284	total: 65.7ms	remaining: 3.8s
17:	learn: 0.3987404	total: 68.7ms	remaining: 3.75s
18:	learn: 0.3899205	total: 72ms	remaining: 3.7

217:	learn: 0.1985720	total: 719ms	remaining: 2.58s
218:	learn: 0.1985103	total: 721ms	remaining: 2.57s
219:	learn: 0.1982778	total: 724ms	remaining: 2.57s
220:	learn: 0.1981356	total: 727ms	remaining: 2.56s
221:	learn: 0.1980137	total: 730ms	remaining: 2.56s
222:	learn: 0.1978788	total: 734ms	remaining: 2.56s
223:	learn: 0.1977095	total: 738ms	remaining: 2.56s
224:	learn: 0.1975582	total: 741ms	remaining: 2.55s
225:	learn: 0.1973169	total: 745ms	remaining: 2.55s
226:	learn: 0.1971925	total: 748ms	remaining: 2.55s
227:	learn: 0.1970265	total: 752ms	remaining: 2.55s
228:	learn: 0.1969239	total: 756ms	remaining: 2.54s
229:	learn: 0.1967662	total: 759ms	remaining: 2.54s
230:	learn: 0.1965452	total: 762ms	remaining: 2.54s
231:	learn: 0.1964935	total: 766ms	remaining: 2.53s
232:	learn: 0.1962787	total: 769ms	remaining: 2.53s
233:	learn: 0.1961988	total: 772ms	remaining: 2.53s
234:	learn: 0.1959115	total: 775ms	remaining: 2.52s
235:	learn: 0.1958046	total: 778ms	remaining: 2.52s
236:	learn: 

403:	learn: 0.1754814	total: 1.32s	remaining: 1.94s
404:	learn: 0.1752830	total: 1.32s	remaining: 1.94s
405:	learn: 0.1751131	total: 1.32s	remaining: 1.94s
406:	learn: 0.1749913	total: 1.33s	remaining: 1.93s
407:	learn: 0.1747162	total: 1.33s	remaining: 1.93s
408:	learn: 0.1746066	total: 1.33s	remaining: 1.93s
409:	learn: 0.1745145	total: 1.34s	remaining: 1.92s
410:	learn: 0.1743779	total: 1.34s	remaining: 1.92s
411:	learn: 0.1742202	total: 1.34s	remaining: 1.92s
412:	learn: 0.1740669	total: 1.35s	remaining: 1.91s
413:	learn: 0.1739604	total: 1.35s	remaining: 1.91s
414:	learn: 0.1737077	total: 1.35s	remaining: 1.91s
415:	learn: 0.1735811	total: 1.36s	remaining: 1.91s
416:	learn: 0.1734919	total: 1.36s	remaining: 1.9s
417:	learn: 0.1733790	total: 1.36s	remaining: 1.9s
418:	learn: 0.1733095	total: 1.37s	remaining: 1.9s
419:	learn: 0.1732163	total: 1.37s	remaining: 1.89s
420:	learn: 0.1731051	total: 1.37s	remaining: 1.89s
421:	learn: 0.1729426	total: 1.38s	remaining: 1.89s
422:	learn: 0.1

595:	learn: 0.1532797	total: 1.94s	remaining: 1.32s
596:	learn: 0.1531962	total: 1.95s	remaining: 1.31s
597:	learn: 0.1530854	total: 1.95s	remaining: 1.31s
598:	learn: 0.1530150	total: 1.95s	remaining: 1.31s
599:	learn: 0.1529676	total: 1.96s	remaining: 1.3s
600:	learn: 0.1528453	total: 1.96s	remaining: 1.3s
601:	learn: 0.1528027	total: 1.96s	remaining: 1.3s
602:	learn: 0.1526745	total: 1.97s	remaining: 1.29s
603:	learn: 0.1525520	total: 1.97s	remaining: 1.29s
604:	learn: 0.1524728	total: 1.97s	remaining: 1.29s
605:	learn: 0.1524241	total: 1.98s	remaining: 1.29s
606:	learn: 0.1523449	total: 1.98s	remaining: 1.28s
607:	learn: 0.1521713	total: 1.99s	remaining: 1.28s
608:	learn: 0.1519574	total: 1.99s	remaining: 1.28s
609:	learn: 0.1518261	total: 2s	remaining: 1.28s
610:	learn: 0.1518190	total: 2s	remaining: 1.27s
611:	learn: 0.1516988	total: 2s	remaining: 1.27s
612:	learn: 0.1516341	total: 2.01s	remaining: 1.27s
613:	learn: 0.1515374	total: 2.01s	remaining: 1.26s
614:	learn: 0.1514381	to

784:	learn: 0.1345724	total: 2.56s	remaining: 702ms
785:	learn: 0.1344781	total: 2.56s	remaining: 699ms
786:	learn: 0.1343893	total: 2.57s	remaining: 695ms
787:	learn: 0.1342760	total: 2.57s	remaining: 692ms
788:	learn: 0.1341523	total: 2.58s	remaining: 689ms
789:	learn: 0.1340898	total: 2.58s	remaining: 686ms
790:	learn: 0.1340520	total: 2.58s	remaining: 683ms
791:	learn: 0.1339555	total: 2.59s	remaining: 680ms
792:	learn: 0.1338917	total: 2.59s	remaining: 677ms
793:	learn: 0.1338755	total: 2.6s	remaining: 674ms
794:	learn: 0.1337761	total: 2.6s	remaining: 670ms
795:	learn: 0.1336979	total: 2.6s	remaining: 667ms
796:	learn: 0.1335511	total: 2.61s	remaining: 664ms
797:	learn: 0.1334458	total: 2.61s	remaining: 661ms
798:	learn: 0.1333669	total: 2.61s	remaining: 657ms
799:	learn: 0.1332128	total: 2.62s	remaining: 654ms
800:	learn: 0.1331596	total: 2.62s	remaining: 651ms
801:	learn: 0.1331144	total: 2.63s	remaining: 648ms
802:	learn: 0.1330146	total: 2.63s	remaining: 645ms
803:	learn: 0.1

...

339:	learn: 0.1709753	total: 1.02s	remaining: 1.99s
340:	learn: 0.1708727	total: 1.04s	remaining: 2.01s
341:	learn: 0.1707034	total: 1.04s	remaining: 2.01s
342:	learn: 0.1706188	total: 1.05s	remaining: 2s
343:	learn: 0.1705380	total: 1.05s	remaining: 2s
344:	learn: 0.1703059	total: 1.05s	remaining: 2s
345:	learn: 0.1702213	total: 1.05s	remaining: 1.99s
346:	learn: 0.1700960	total: 1.06s	remaining: 1.99s
347:	learn: 0.1699465	total: 1.06s	remaining: 1.99s
348:	learn: 0.1698342	total: 1.06s	remaining: 1.98s
349:	learn: 0.1697022	total: 1.07s	remaining: 1.98s
350:	learn: 0.1695576	total: 1.07s	remaining: 1.98s
351:	learn: 0.1693781	total: 1.07s	remaining: 1.97s
352:	learn: 0.1693062	total: 1.07s	remaining: 1.97s
353:	learn: 0.1692214	total: 1.08s	remaining: 1.97s
354:	learn: 0.1690783	total: 1.08s	remaining: 1.96s
355:	learn: 0.1689659	total: 1.08s	remaining: 1.96s
356:	learn: 0.1688577	total: 1.09s	remaining: 1.96s
357:	learn: 0.1687414	total: 1.09s	remaining: 1.95s
358:	learn: 0.1685101

543:	learn: 0.1433475	total: 1.62s	remaining: 1.36s
544:	learn: 0.1431725	total: 1.62s	remaining: 1.35s
545:	learn: 0.1430301	total: 1.63s	remaining: 1.35s
546:	learn: 0.1429592	total: 1.63s	remaining: 1.35s
547:	learn: 0.1428396	total: 1.63s	remaining: 1.35s
548:	learn: 0.1426851	total: 1.64s	remaining: 1.34s
549:	learn: 0.1425165	total: 1.64s	remaining: 1.34s
550:	learn: 0.1424184	total: 1.64s	remaining: 1.34s
551:	learn: 0.1423727	total: 1.65s	remaining: 1.34s
552:	learn: 0.1423344	total: 1.65s	remaining: 1.33s
553:	learn: 0.1422070	total: 1.65s	remaining: 1.33s
554:	learn: 0.1421012	total: 1.66s	remaining: 1.33s
555:	learn: 0.1420214	total: 1.66s	remaining: 1.33s
556:	learn: 0.1418598	total: 1.66s	remaining: 1.32s
557:	learn: 0.1417275	total: 1.67s	remaining: 1.32s
558:	learn: 0.1416627	total: 1.67s	remaining: 1.32s
559:	learn: 0.1415673	total: 1.67s	remaining: 1.31s
560:	learn: 0.1415542	total: 1.68s	remaining: 1.31s
561:	learn: 0.1415412	total: 1.68s	remaining: 1.31s
562:	learn: 

716:	learn: 0.1253753	total: 2.23s	remaining: 879ms
717:	learn: 0.1252215	total: 2.23s	remaining: 876ms
718:	learn: 0.1251522	total: 2.23s	remaining: 873ms
719:	learn: 0.1250681	total: 2.24s	remaining: 871ms
720:	learn: 0.1249453	total: 2.24s	remaining: 868ms
721:	learn: 0.1248260	total: 2.25s	remaining: 865ms
722:	learn: 0.1247391	total: 2.25s	remaining: 863ms
723:	learn: 0.1245927	total: 2.26s	remaining: 860ms
724:	learn: 0.1244776	total: 2.26s	remaining: 857ms
725:	learn: 0.1244261	total: 2.26s	remaining: 855ms
726:	learn: 0.1243000	total: 2.27s	remaining: 852ms
727:	learn: 0.1242537	total: 2.27s	remaining: 850ms
728:	learn: 0.1241502	total: 2.28s	remaining: 848ms
729:	learn: 0.1240648	total: 2.28s	remaining: 845ms
730:	learn: 0.1239653	total: 2.29s	remaining: 842ms
731:	learn: 0.1238818	total: 2.29s	remaining: 840ms
732:	learn: 0.1238088	total: 2.3s	remaining: 837ms
733:	learn: 0.1237931	total: 2.3s	remaining: 834ms
734:	learn: 0.1236671	total: 2.31s	remaining: 831ms
735:	learn: 0.

891:	learn: 0.1095754	total: 2.84s	remaining: 344ms
892:	learn: 0.1094918	total: 2.85s	remaining: 341ms
893:	learn: 0.1094416	total: 2.85s	remaining: 338ms
894:	learn: 0.1093482	total: 2.85s	remaining: 335ms
895:	learn: 0.1092741	total: 2.86s	remaining: 332ms
896:	learn: 0.1092245	total: 2.86s	remaining: 329ms
897:	learn: 0.1091339	total: 2.87s	remaining: 326ms
898:	learn: 0.1091281	total: 2.87s	remaining: 323ms
899:	learn: 0.1090474	total: 2.87s	remaining: 319ms
900:	learn: 0.1089574	total: 2.88s	remaining: 316ms
901:	learn: 0.1088562	total: 2.88s	remaining: 313ms
902:	learn: 0.1087704	total: 2.88s	remaining: 310ms
903:	learn: 0.1087465	total: 2.89s	remaining: 307ms
904:	learn: 0.1086483	total: 2.89s	remaining: 303ms
905:	learn: 0.1085402	total: 2.89s	remaining: 300ms
906:	learn: 0.1084570	total: 2.9s	remaining: 297ms
907:	learn: 0.1083471	total: 2.9s	remaining: 294ms
908:	learn: 0.1081829	total: 2.9s	remaining: 291ms
909:	learn: 0.1080780	total: 2.91s	remaining: 288ms
910:	learn: 0.1

StackingClassifier(estimators=[('rf', RandomForestClassifier()),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constrain...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None,
                                              reg_alpha=None, reg_lambda=None, ...)),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x000001BF2F4205B0>),
                               ('lightgbm', LGBMClassifier())])

In [43]:
y_preds = stacking.predict(X_test)

In [44]:
print(confusion_matrix(y_preds, y_test))

[[5023  416]
 [ 188  538]]


In [45]:
print(classification_report(y_preds, y_test))

              precision    recall  f1-score   support

       False       0.96      0.92      0.94      5439
        True       0.56      0.74      0.64       726

    accuracy                           0.90      6165
   macro avg       0.76      0.83      0.79      6165
weighted avg       0.92      0.90      0.91      6165



In [150]:
df_copy = df.copy()
df_copy.to_csv('file_touse_forH2o.csv', index=False)
data = h2o.import_file("file_touse_forH2o.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [151]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

x = data.columns[:-1]
y = data.columns[-1]

train, test = data.split_frame(ratios=[0.8])

automl = H2OAutoML(max_models=10, seed=42)
automl.train(x=x, y=y, training_frame=train)
best_model = automl.leader
predictions = best_model.predict(test)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,8 mins 02 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_lenovo_t223or
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.792 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


AutoML progress: |█
15:05:32.360: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [152]:
best_model

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_1_AutoML_3_20230516_150532


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    56                 56                          41625                  10           15           13.7857       28            62            53.9464

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.053080274807638224
RMSE: 0.23039156843868705
LogLoss: 0.1748686091908504
Mean Per-Class Error: 0.12720423358125285
AUC: 0.9660816476384481
AUCPR: 0.8495838295881576
Gini: 0.9321632952768961

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.32896421823400857
       False    True    Error    Rate
-----  -------  ------  -------  --------------
False  7861     499     0.0597   (499.0/8360.0)
True   295      1220    0.1947   (295.0/1515.0)
Total  8156     1719    0.0804   (794.0/9875.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.328964     0.754484  207
max f2                       0.149509     0.831894  283
max f0point5                 0.510612     0.789474  146
max accuracy                 0.476291     0.927392  160
max precision                0.978917     1         0
max recall                   0.0251155    1         366
max specificity              0.978917     1         0
max absolute_mcc             0.443985     0.70955   170
max min_per_class_accuracy   0.214384     0.896292  252
max mean_per_class_accuracy  0.149509     0.906324  283
max tns                      0.978917     8360      0
max fns                      0.978917     1512      0
max fps                      0.00148279   8360      399
max tps                      0.0251155    1515      366
max tnr                      0.978917     1         0
max fnr                      0.978917     0.99802   0
max fpr                      0.00148279   1         399
max tpr                      0.0251155    1         366

Gains/Lift Table: Avg response rate: 15.34 %, avg score: 15.37 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100253                   0.944536           6.51815    6.51815            1                0.958029    1                           0.958029            0.0653465       0.0653465                  551.815   551.815            0.0653465
2        0.0200506                   0.914794           6.45231    6.48523            0.989899         0.929917    0.994949                    0.943973            0.0646865       0.130033                   545.231   548.523            0.129913
3        0.0300759                   0.880812           6.45231    6.47426            0.989899         0.897682    0.993266                    0.928543            0.0646865       0.194719                   545.231   547.426            0.19448
4        0.04                        0.84511            6.11908    6.38614            0.938776         0.863892    0.979747                    0.912503            0.0607261       0.255446                   511.908   538.614            0.254489
5        0.0500253                   0.800584           5.85975    6.28065            0.89899          0.8218

In [162]:
best_model = automl.leaderboard
best_model

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_1_AutoML_3_20230516_150532,0.930878,0.225636,0.7487,0.162567,0.26206,0.0686754
StackedEnsemble_AllModels_1_AutoML_3_20230516_150532,0.930703,0.224448,0.753697,0.183188,0.261181,0.0682157
StackedEnsemble_BestOfFamily_1_AutoML_3_20230516_150532,0.929976,0.22575,0.748409,0.176876,0.26209,0.0686912
GBM_2_AutoML_3_20230516_150532,0.929543,0.228564,0.74847,0.193906,0.263266,0.0693088
GBM_5_AutoML_3_20230516_150532,0.927875,0.231148,0.740803,0.184621,0.265169,0.0703148
GBM_3_AutoML_3_20230516_150532,0.927159,0.232031,0.742248,0.178601,0.265549,0.0705165
GBM_grid_1_AutoML_3_20230516_150532_model_1,0.926447,0.233852,0.738862,0.175356,0.265879,0.0706918
GBM_4_AutoML_3_20230516_150532,0.925791,0.236675,0.728301,0.184453,0.26889,0.072302
XRT_1_AutoML_3_20230516_150532,0.918985,0.264114,0.73718,0.189512,0.266353,0.070944
DRF_1_AutoML_3_20230516_150532,0.918947,0.292101,0.737294,0.186566,0.266393,0.0709654


In [160]:
automl.head(rows=automl.nrows)

AttributeError: 'H2OAutoML' object has no attribute 'head'

In [163]:
m = h2o.get_model("GBM_1_AutoML_3_20230516_150532")

In [165]:
m = h2o.get_model("StackedEnsemble_AllModels_1_AutoML_3_20230516_150532")

In [166]:
m

key,value
Stacking strategy,cross_validation
Number of base models (used / total),7/10
# GBM base models (used / total),5/6
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),0/1
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [1]:
!pip install git+https://github.com/hyperopt/hyperopt-sklearn.git

  Cloning https://github.com/hyperopt/hyperopt-sklearn.git to c:\users\lenovo\appdata\local\temp\pip-req-build-13kr5w_8


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: C:\Users\lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [46]:
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [17]:
X_train

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates in %,ExitRates in %,PageValues,SpecialDay (probability),Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
0,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,-0.308821,2,-1.233426,-0.790293,-0.894178,-0.762629,2,-0.550552
1,-0.696993,-0.457191,-0.396478,-0.244931,-0.668518,-0.590903,-0.457683,1.171473,-0.317178,-0.308821,2,-0.136078,-0.207952,-0.894178,-0.514182,2,-0.550552
5,-0.696993,-0.457191,-0.396478,-0.244931,-0.286270,-0.543757,-0.132035,-0.380935,-0.317178,-0.308821,2,-0.136078,-0.207952,-0.894178,-0.265735,2,-0.550552
6,-0.696993,-0.457191,-0.396478,-0.244931,-0.691003,-0.624348,3.667189,3.229316,-0.317178,1.702146,2,-0.136078,0.956730,-0.061364,-0.265735,2,-0.550552
7,-0.395938,-0.457191,-0.396478,-0.244931,-0.713488,-0.624348,3.667189,3.229316,-0.317178,-0.308821,2,-1.233426,-0.207952,-0.894178,0.231158,2,1.816360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.206173,0.363075,-0.396478,-0.244931,0.478227,0.307822,-0.310366,-0.288966,0.342125,-0.308821,1,2.058618,2.121412,-0.894178,-0.762629,2,1.816360
12326,-0.696993,-0.457191,-0.396478,-0.244931,-0.601062,-0.380957,-0.457683,-0.447364,-0.317178,-0.308821,7,0.961270,-0.207952,-0.894178,0.976499,2,1.816360
12327,-0.696993,-0.457191,-0.396478,-0.244931,-0.578577,-0.528063,1.261014,0.897093,-0.317178,-0.308821,7,0.961270,-0.207952,-0.894178,2.218733,2,1.816360
12328,0.507228,-0.032916,-0.396478,-0.244931,-0.376210,-0.443536,-0.457683,-0.453140,-0.317178,-0.308821,7,-0.136078,-0.207952,-0.061364,1.721839,2,-0.550552


In [47]:
hyperopt = HyperoptEstimator(classifier=any_classifier('cla'), preprocessing=any_preprocessing('pre'),
                             algo=tpe.suggest, max_evals=50, trial_timeout=30)
# Fit the training data
hyperopt.fit(X_train.values, y_train.values)

  0%|                                                                            | 0/1 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'

In [26]:
!pip install autokeras

     -------------------------------------- 148.6/148.6 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 176.1/176.1 kB 5.4 MB/s eta 0:00:00
     -------------------------------------- 527.7/527.7 kB 6.6 MB/s eta 0:00:00
     ---------------------------------------- 5.0/5.0 MB 3.5 MB/s eta 0:00:00
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl (895 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
     -------------------------------------- 100.6/100.6 kB 5.6 MB/s eta 0:00:00
     -------------------------------------- 455.9/455.9 MB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 5.9/5.9 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 438.7/438.7 kB 5.4 MB/s eta 0:00:00
     --------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ortools 9.6.2534 requires protobuf>=4.21.12, but you have protobuf 3.19.6 which is incompatible.

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: C:\Users\lenovo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [48]:
import autokeras
# Create the model
keras = autokeras.StructuredDataClassifier(max_trials=8)
# Fit the training dataset
keras.fit(X_train, y_train, epochs=100)
# Predict the validation data
pred_keras = keras.predict(X_test)
# Compute the accuracy
print('Accuracy: ' + str(accuracy_score(y_test, pred_keras)))

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
keras_export = keras.export_model()
keras_export.summary()

In [37]:
!pip uninstall mljar-supervised

^C


In [36]:
from supervised.automl import AutoML
# Create the model
mljar = AutoML(mode="Compete",  eval_metric="log_loss", total_time_limit=300,
               features_selection=True)
# Fit the training data
mljar.fit(X_train, y_train)
# Predict the training data
mljar_pred = mljar.predict(X_val)

ImportError: cannot import name '_safe_assign' from 'sklearn.utils' (C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\__init__.py)